If cuda is not working first try reinstalling torch and restarting computer, if this does not work you are fucked yet again

In [1]:

import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device
torch.cuda.is_available()

True

In [1]:
# Import necessary libraries
from top2vec import Top2Vec


## top2vec on news data

In [4]:
%%time
import pandas as pd
ROOT_PATH = '/home/jhladnik/'

NUM_SAMPLES = 1000
df = pd.read_parquet(
    f'{ROOT_PATH}/data/eventregistry/df_news_lemmas_{NUM_SAMPLES}.parquet.gzip')


CPU times: user 124 ms, sys: 31.8 ms, total: 156 ms
Wall time: 104 ms


In [41]:
import time
from gensim.corpora.dictionary import Dictionary
start = time.time()
dictionary = Dictionary(list(df['lemmatized_text']))
corpus = [dictionary.doc2bow(text) for text in list(df['lemmatized_text'])]

def join_lemmas(list_lemmas):
    return ' '.join(list_lemmas)
df['lemmatized_body'] = df['lemmatized_text'].apply(join_lemmas)
# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token
results_dict = {"model_name": [], "num_topics": [],
"topic_diversity": [], "topic_coherence_npmi": [], "topic_coherence_umass":[], "seed":[]}

top_n_words = 10

In [ ]:
num_topics_params = [5]#, 8, 10, 15, 20, 25, 30]
seeds = [1]#, 2, 3]
for num_topics in tqdm.tqdm(num_topics_params):
    for seed in seeds:

In [5]:
# 3 min on remote  for 5k samples
# 30s on remote for 1k samples
trained_model = Top2Vec(documents=list(df["lemmatized_body"]), speed="learn", workers=24)


2023-01-29 14:09:46,859 - top2vec - INFO - Pre-processing documents for training
2023-01-29 14:09:47,300 - top2vec - INFO - Creating joint document/word embedding
2023-01-29 14:10:00,125 - top2vec - INFO - Creating lower dimension embedding of documents
2023-01-29 14:10:13,890 - top2vec - INFO - Finding dense areas of documents
2023-01-29 14:10:13,914 - top2vec - INFO - Finding topics


In [62]:
def get_top_words_for_topics_top2vec(trained_model, top_n_words, num_topics=15):
    topic_words, word_scores, topic_nums = model.get_topics()
    num_identified_topics = len(topic_words)
    if num_identified_topics < num_topics:
        topic_representations = topic_words
        print(f"identified {num_identified_topics} topics, but {num_topics} were requested")
    else:
        topic_mapping = trained_model.hierarchical_topic_reduction(num_topics=num_topics)
        topic_representations = model.topic_words_reduced
    
    topic_representations = [topic_words[:top_n_words] for topic_words in topic_representations]
    return topic_representations

top_n_words = 10
num_topics = 6
trained_model = model
topics_representations = get_top_words_for_topics_top2vec(model, top_n_words=top_n_words, num_topics=num_topics)


In [64]:
from evaluation import topic_diversity, topic_coherence
results_dict["model_name"].append("top2vec")
results_dict["num_topics"].append(num_topics)
results_dict["topic_diversity"].append(topic_diversity(topics_representations, top_n_words))
results_dict["topic_coherence_npmi"].append(topic_coherence(df, trained_model, topics_representations, top_k_words=top_n_words, coherence_metric="c_npmi"))
results_dict["topic_coherence_umass"].append(topic_coherence(df, trained_model, topics_representations, top_k_words=top_n_words, coherence_metric="u_mass"))
results_dict["seed"].append(seed)

In [65]:
results_dict

{'model_name': ['top2vec', 'top2vec', 'top2vec', 'top2vec'],
 'num_topics': [6, 6, 6],
 'topic_diversity': [1.0, 1.0, 1.0],
 'topic_coherence_npmi': [-0.007363732517235069, -0.007363732517235069],
 'topic_coherence_umass': [-1.824472197745601, -1.824472197745601],
 'seed': []}

## top2vec on tweets data

In [5]:
import pandas as pd
ROOT_PATH = '/home/jhladnik/'

NUM_SAMPLES = 50000
df = pd.read_parquet(
    f'{ROOT_PATH}/data/sl-tweets/df_tweets_lemmas_{NUM_SAMPLES}.parquet.gzip')


In [6]:
df["lemmatized_text_string"] = df.lemmatized_text.apply(lambda x: " ".join(x))

In [7]:
%%time
# 6 min on remote  for 50k samples
model = Top2Vec(documents=list(df["lemmatized_text_string"]), speed="learn", workers=24)


2023-01-24 20:35:20,369 - top2vec - INFO - Pre-processing documents for training
2023-01-24 20:35:21,929 - top2vec - INFO - Creating joint document/word embedding
2023-01-24 20:41:09,343 - top2vec - INFO - Creating lower dimension embedding of documents
2023-01-24 20:41:55,396 - top2vec - INFO - Finding dense areas of documents
2023-01-24 20:41:58,815 - top2vec - INFO - Finding topics


CPU times: user 24min 12s, sys: 3min 28s, total: 27min 41s
Wall time: 6min 38s


In [8]:
topic_words, word_scores, topic_nums = model.get_topics()

In [9]:
import numpy as np
from evaluation import get_top_words_for_topics_bertopic, topic_diversity

topics_representations = topic_words
#topics_representations = get_top_words_for_topics_bertopic(topic_model, 25)


### calculate topic diversity

In [10]:
topics_representations.shape

(160, 50)

In [11]:


topic_diversity(topics_representations, 25)

0.21225

### calcualte topic coherence

In [12]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary



topic_coherence(None, topics_representations, top_k_words=10)

-0.41547616656131436